# **Uniform Symplectic Matroids**

**Theorem** : ***uniform symplectic matroids***

$(J, -, J_k)$ is symplectic matroids.

**Remark** : For detail, please see my graduation thesis.

We remember the lemma, 

**Lemma** : ***trivial representation***

We assume $\text{rank}A = k$ and $D \in \mathbb{F}^{n \times n}$ is diagonal matrix. Then, $(A \mid AD)$ is representation of symplectic matroids.

then, we get the next theorem.

**Theorem** : ***representability of uniform symplectic matroids***

On the field $\mathbb{F}$, uniform symplectic matroids $(J, -, J_k)$ is representable if and only if uniform matroids $U_{k, n}$ is representable.

For detail proof, please see my graduation thesis. 

Concicely speaking, we set $A$ is representation of uniform matroid. i.e. for all k-row vectors of $A$ is linearly independent. then $(A \mid AD)$ represent uniform symplectic matroids. Then we got *trivial* representable uniform symplectic matroids.

However there may be *non trivial* representation of uniform symplectic matroids. Let's get non trivial representation.

We notice the next corollary.

**corollary** : 

If uniform symplectic matroids is represented by $(A \mid B)$, $A,\ B$ is representation of uniform matroids.

firstly, we will find all of representation of $U_{k, n}$

**Proof**:

All of $k$-subsets of $A,\ B$ are included in all of admissible subset of $(A \mid B)$

In [15]:
import numpy as np
import itertools
import random

import symplectic_matroids

# F_q
q = 5 

# J = [n]
n = 4 

# size of basis
k = 2 

determinant algorithm return precise value

In [16]:
def det(M):
    M = [row[:] for row in M] # make a copy to keep original M unmodified
    N, sign, prev = len(M), 1, 1
    for i in range(N-1):
        if M[i][i] == 0: # swap with another row having nonzero i's elem
            swapto = next( (j for j in range(i+1,N) if M[j][i] != 0), None )
            if swapto is None:
                return 0 # all M[*][i] are zero => zero determinant
            M[i], M[swapto], sign = M[swapto], M[i], -sign
        for j in range(i+1,N):
            for k in range(i+1,N):
                assert ( M[j][k] * M[i][i] - M[j][i] * M[i][k] ) % prev == 0
                M[j][k] = ( M[j][k] * M[i][i] - M[j][i] * M[i][k] ) // prev
        prev = M[i][i]
    return sign * M[-1][-1]

## reference : (https://stackoverflow.com/questions/66192894/precise-determinant-of-integer-nxn-matrix)

### all of deteminant of $k \times k$ matrix

For reducing computing complexity, we concider set of corresponding index instead of matrix.

We define 

d_det : (index_1, index_2, $\cdots$ index_k) $\mapsto$ det(row_1 row_2 $\cdots$ row_k)

For example, on $F = \mathbb{F}_2$

$(list)\ F_3 = 
\begin{bmatrix}
  \begin{bmatrix}
  0\\
  0\\
  0
  \end{bmatrix}
  \begin{bmatrix}
  1\\
  0\\
  0
  \end{bmatrix}
  \begin{bmatrix}
  0\\
  1\\
  0
  \end{bmatrix}
  \begin{bmatrix}
  1\\
  1\\
  0
  \end{bmatrix}
  \begin{bmatrix}
  0\\
  0\\
  1
  \end{bmatrix}
  \begin{bmatrix}
  1\\
  0\\
  1
  \end{bmatrix}
  \begin{bmatrix}
  0\\
  1\\
  1
  \end{bmatrix}
  \begin{bmatrix}
  1\\
  1\\
  1
  \end{bmatrix}
\end{bmatrix}
$

d_det( ( 2, 3, 5 ) ) = 
$
\begin{bmatrix}
0 && 1 && 1\\
1 && 1 && 0\\
0 && 0 && 1
\end{bmatrix}
$
(0-indexed)

In [17]:
Fk = list(itertools.product(range(q), repeat=k))
Fk = np.array(list(map(lambda x:np.array(x), Fk)))

d_det = dict()
for t_idx in itertools.combinations_with_replacement(range(len(Fk)), k):
    d_det[t_idx] = det(Fk[list(t_idx)])%q

### all of $U_{k, n}$

In [18]:
# if k >= 1, the representation of Ukn don't have 0 vector. 
matrices = itertools.combinations_with_replacement(range(len(Fk[1:])),n)
matrices = list(map(lambda x:np.array(x) , matrices))

subset_indeces = tuple(itertools.combinations(range(n),k))

# virtual matrices (only having Fk index number)
v_U_kn = list()
for matrix in matrices:
    flag = True
    for idx in subset_indeces:
        if d_det[tuple(matrix[list(idx)])] == 0:
            flag = False
            break
    if flag:
        v_U_kn.append(matrix)

In [19]:
randint = random.randrange(len(v_U_kn))

print("the number of representation of Ukn")
print(len(v_U_kn))
print("example")
print(Fk[list(v_U_kn[randint])].T)

the number of representation of Ukn
3200
example
[[0 3 3 4]
 [3 0 2 3]]


Nextly, we check $AB^T == BA^T$. This is because before theorem

**Theorem** : *representable symplectic matroids*

We assume $A = (\bm{a}_1\ \cdots\ \bm{a}_n), B = (\bm{b}_1\ \cdots\ \bm{b}_n) \in \mathbb{F}_{q}^{k \times n},\ \text{rank}(A \mid B) = k,\ AB^T = BA^T$, 

$i \in \{1, \cdots, n\}$ column index $i$ means $\bm{a}_i$ index $-i$ means $\bm{b}_i$.

We define $\mathcal{B} = \{B \in J_k \mid \forall X \in J_k,\ \text{det}(A \mid B)[X] \neq 0\}$, then $(J,\ -,\ \mathcal{B})$ is symplectic matroids.

### indeces to matrices

For calculating $AB^T == BA^T$.

In [20]:
U_kn = [np.array([np.array([0 for _ in range(k)]) for i in range(n)]) for j in range(len(v_U_kn))]

for u in range(len(U_kn)):
    for idx in range(n):
        U_kn[u][idx] = Fk[v_U_kn[u][idx]]

In [21]:
print(len(U_kn))
print(U_kn[randint].T)

3200
[[0 3 3 4]
 [3 0 2 3]]


### check $AB^T =BA^T$

In [22]:
candidate0 = list()
for idxA, idxB in itertools.combinations(range(len(U_kn)), 2):
    ABt = (U_kn[idxA].T@U_kn[idxB])%q
    if np.array_equal(ABt, ABt.T):
        candidate0.append((v_U_kn[idxA],v_U_kn[idxB]))

In [23]:
randint = random.randrange(len(candidate0))

print("A")
print(Fk[list(candidate0[randint][0])].T)

print("B")
print(Fk[list(candidate0[randint][1])].T)

print("ABt")
print(Fk[list(candidate0[randint][0])].T@Fk[list(candidate0[randint][1])]%q)

print(len(candidate0))

A
[[0 2 4 4]
 [4 2 1 3]]
B
[[1 2 2 2]
 [1 0 1 3]]
ABt
[[0 1]
 [1 4]]
1033917


### find non trivial representation

In [24]:
non_trivial = list()
for A, B in candidate0:
    At = Fk[A]
    Bt = Fk[B]

    flag0 = False # there exists row_i and row_{-i} s.t. they are linearly independent.
    for i in range(n):
        flag1 = True # for all (scalar) k in F_q, a_i != k*b_i
        for scalar in range(q):
            if np.array_equal(At[i],scalar*Bt[i]%q):
               flag1 = False
               break

        if flag1:
            flag0 = True
            break
    
    if flag0:
        non_trivial.append((A,B))

In [25]:
randint = random.randrange(len(non_trivial))

A, B = non_trivial[randint]
print("A")
print(Fk[A].T)
print("B")
print(Fk[B].T)
print("ABt")
print(Fk[A].T@Fk[B]%q)

A
[[0 1 2 4]
 [3 0 4 1]]
B
[[0 1 1 4]
 [4 2 3 0]]
ABt
[[4 3]
 [3 4]]


In [26]:
print("the number of found nontrivial representation")
print(len(non_trivial))
print("the proportion of nontrivial")
print(len(non_trivial)/len(candidate0))

the number of found nontrivial representation
984209
the proportion of nontrivial
0.9519226398250537


## **Conclusion**

There exists nontrivial representation of uniform symplectic matroids.

The caluculated  the number of nontrivial representation is not precise because matrix swppped the row isn't concider in calculation. (for calculation complexity, we only used combination of row from Fk. Not permutation.) However, the proportion of nontrivial is so high. I think there may be another representation of uniform symplectic matroids. (For example, using *special* regular matrix $P$, $(A \mid AP)$ )